In [ ]:
import torch as T
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import scipy.linalg
from functools import partial
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import scipy.sparse as sparse
from scipy import stats
from __future__ import print_function
import gc
from tqdm import tqdm
from __future__ import division
import pandas as pd
import seaborn as sb
import seaborn as sns
import os
from tqdm import tqdm
from scipy.spatial import distance
from scipy.cluster import hierarchy
from pandas import DataFrame

In [ ]:
import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

import passivity_2p_imaging_utils as p2putils
import visualization_utils as vizutil
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

f = all_data['e'][2]

data = np.load("../cnmf_f01555.npz")
cnmf = data['cnmf'].astype(np.float32)
raw = data['raw'].astype(np.float32)
del data
# cnmf_t = T.from_numpy(raw).cuda()
neurons = raw
# threshold = np.sort(neurons.std(0))[-5001] # delete me

neuron_ids = np.sort(np.argsort(neurons.std(0))[-5000:])
neurons = neurons[:,neuron_ids]

# df, sig = f.get_cnmf_roi_table_and_signals()
df = f.get_roi_table()
df = df.iloc[neuron_ids]
df.reset_index(drop=True, inplace=True)


ortho_regions = [u'in_l_cerebellum', u'in_r_cerebellum', 
    u'in_l_vthal', u'in_r_vthal', u'in_l_tectum', u'in_r_tectum', 
    u'in_l_raphe', u'in_r_raphe', u'in_l_hind', u'in_r_hind', 
    u'in_l_dthal', u'in_r_dthal', u'in_l_LHb', u'in_r_LHb', 
    u'in_l_tel', u'in_r_tel', u'in_l_MHb',  u'in_r_MHb']

regions = []
for r in ortho_regions:
    if r in df.columns:
        regions.append(r)

def moving_average(a, n=3):
    # a is a signal
    ret = np.cumsum(a,0) # sum over time
    ret[n:] = ret[n:] - ret[:-n] # diff of n samples back
    rm = ret[n - 1:] / n
    pad_start = np.full((n-1,rm.shape[1]), rm[0])
    return np.vstack([pad_start, rm])

def ewma(data,span):
    "exponential weighted moving average."
    df = DataFrame(data)
    return df.ewm(span).mean().values

def df_f(x,ma_window=6,span=6):
    u = moving_average(x,ma_window)
    return ewma((x - u)/u, span)

neurons = T.from_numpy(df_f(neurons,1000,6).astype(np.float32)).cuda()
# neurons = T.from_numpy(df_f(neurons).astype(np.float32)).cuda()
neurons_norm = (neurons - neurons.mean(0))/(neurons.std(0)+1e-8)
neurons = neurons_norm
print(neurons.shape)

In [ ]:
df.columns[np.where([c in ortho_regions for c in df.columns])]

In [ ]:
raw.shape

In [ ]:
plt.plot(raw[:500,0])

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(raw[:,[30]])

In [ ]:
i = 120

plt.figure(figsize=(20,5))
plt.plot(df_f(raw[:,[i]],2000,3))

plt.figure(figsize=(20,5))
plt.plot(df_f(raw[:,[i]],1000,6))

plt.figure(figsize=(20,5))
plt.plot(df_f(raw[:,[i]],100,6))

plt.figure(figsize=(20,5))
plt.plot(df_f(raw[:,[i]],6,6))

In [ ]:
def corrcoef(x):
    "cross-correlation with pytorch"
    # calculate covariance matrix of rows
    mean_x = torch.mean(x, 1, keepdim=True)
    xm = x.sub(mean_x.expand_as(x))
    c = xm.mm(xm.t())
    c = c / (x.size(1) - 1)

    # normalize covariance matrix
    d = torch.diag(c)
    stddev = torch.pow(d, 0.5)
    c = c.div(stddev.expand_as(c)+1e-8)
    c = c.div(stddev.expand_as(c).t()+1e-8)

    # clamp between -1 and 1
    # probably not necessary but numpy does it
    c = torch.clamp(c, -1.0, 1.0)

    return c

# cluster on cross-correlation

neuron_corr = corrcoef(neurons.t())

In [ ]:
neuron_nclust = 5
neuron_linkage = hierarchy.linkage(
    distance.pdist(neuron_corr), method='ward')
neuron_linkage_clusters = hierarchy.fcluster(neuron_linkage,neuron_nclust,criterion='maxclust')

In [ ]:
dtype = np.float32
# x_fish = F.normalize(neurons,0)
x_fish = neurons
time_fish = T.from_numpy(f.frame_st.mean(1).astype(dtype)).cuda()
if dtype==np.float16:
    u_fish = T.cuda.HalfTensor(time_fish.shape).zero_()
    p_fish = T.cuda.HalfTensor(time_fish.shape).zero_()
else:
    u_fish = T.cuda.FloatTensor(time_fish.shape).zero_()
    p_fish = T.cuda.FloatTensor(time_fish.shape).zero_()
u_fish[numpy.searchsorted(f.frame_et[:,-1], f.shock_st,side="left")] = 1

c, coph_dists = hierarchy.cophenet(neuron_linkage, distance.pdist(neurons))
c

In [ ]:
class Granger(nn.Module):
    def __init__(self, nfeatures, n_past_steps, std=0.1, dtype=T.float32,bias=True):
        # we will perform nfeatures x nfeatures regressions
        super(Granger, self).__init__()
        if dtype==T.float32:
            tensor = T.cuda.FloatTensor
        elif dtype==T.float16:
            tensor = T.cuda.HalfTensor
        # Granger coefficient
        self.G = nn.Parameter(tensor(n_past_steps,nfeatures,nfeatures).normal_(std),
                              requires_grad=True)
        self.auto = nn.Parameter(tensor(n_past_steps,nfeatures,nfeatures).normal_(std),
                              requires_grad=True)
        self.bias = bias
        # Bias matrix
        if self.bias:
            self.B = nn.Parameter(tensor(nfeatures, nfeatures).normal_(std),requires_grad=True)

        self.n_past_steps = n_past_steps
        self.tensor = tensor
        self.dtype = dtype

    def forward(self, x_true, n_past_steps=None):
        if n_past_steps==None:
            n_past_steps = self.n_past_steps
        # nfeatures is n neurons & n_past_steps is lag
        # adding dim to make broadcast work
        # x_true is batch x lag x nfeatures x         1
        # G is          1 x lag x nfeatures x nfeatures
        y = self.G[None] * x_true[:,:,:,None]+ self.auto[None] * x_true[:,:,None,:]
        # sum lag so y is batch x nfeatures x nfeatures
        if self.bias:
            return y.sum(1) + self.B[None]
        else:
            return y.sum(1)

def train(model,data,nepochs=1, l1p=1e-8, l2=0, lr=1e-3,batch_size=16, verbose=True):
    # l1 per parameter
    l1 = l1p / np.prod(T.tensor(model.G.shape).numpy())
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
    if verbose:
        epochs = range(nepochs)
    else:
        loader = dataloader
        epochs = tqdm(range(nepochs))
    for epoch in epochs:
        if verbose:
            loader = tqdm(dataloader)
        for X,Y in loader:
            Y_pred = model.forward(X)
            l = 0
#             for param in model.parameters():
#                 l += l1* param.norm(1) + l2 *param.norm(2)
            if model.bias:
                l = l1*model.G.norm(1) + l1*model.B.norm(1)
            else:
                l = l1*model.G.norm(1)
            loss = F.mse_loss(Y[:,None,:], Y_pred) + l
            optimizer.zero_grad()
            loss.backward()# back props
            optimizer.step()# update the parameters
        if verbose:
            print('epoch {}, loss {}'.format(epoch,loss.data))

# nfeatures = 100
# r = np.array([2,3,5,7,11])
# i = 10
# j = 13
# model = Granger(nfeatures,1)
# model.G.data.zero_()
# model.G.data[0,i,j] = r[0]
# model.auto.data.zero_()
# model.auto.data[0,i,j] = r[1]
# model.B.data.zero_()
# model.B.data[i,j] = r[2]
# xt = T.cuda.FloatTensor(1,nfeatures).zero_()
# xt[0,i] = r[3]
# xt[0,j] = r[4]

# r_true = 2*7+3*11+5
# try:
#     assert model(xt[None,[0]])[0,i,j] == r_true
# except:
#     print(model(xt[None,[0]])[0,i,j], r_true)
#     raise

In [ ]:
model = Granger(x_fish.shape[1],1,bias=False)
# trunc_fish = x_fish[0:400]
# trunc_fish = x_fish[440:840]
# trunc_fish = x_fish[840:1240]
# trunc_fish = x_fish[1240:1640]
# trunc_fish = x_fish[1640:2040]
trunc_fish = x_fish[1640:2440]
# trunc_fish = x_fish[640:840]
# trunc_fish = x_fish[2400:2800]
# trunc_fish = x_fish[2600:2800]
# trunc_fish = x_fish
X = trunc_fish[:-1,None]
Y = trunc_fish[1:,None]
data = T.utils.data.TensorDataset(X,Y)
l1 = 5e-6
# train(model,data,10,l1=l1,batch_size=4) # max(bz)==4 for all neurons
train(model,data,3,l1p=1e-0,lr=1e-1,batch_size=1,verbose=False)
train(model,data,15,l1p=5e-2,lr=1e-3,batch_size=1,verbose=False)
G = model.G.data[0].detach().cpu().numpy()

In [ ]:
plt.imshow(model.auto[0,:100,:100].data.detach().cpu())

T.save(model.state_dict(), "granger_15k_raw_all_time.pt")

In [ ]:
# Clustering Granger

g_linkage = hierarchy.linkage(
    distance.pdist(G), method='ward')

g_nclust = 5
g_clusters = hierarchy.fcluster(g_linkage,g_nclust,criterion='maxclust')
# see question for code prior to "color mapping"
cm_cycle = plt.get_cmap("tab10")
g_cmap = [cm_cycle(i) for i in range(g_nclust)]
neuron_cmap = [cm_cycle(i) for i in range(g_nclust,g_nclust+neuron_nclust)]

In [ ]:
neuron_linkage_clusters

In [ ]:
all_linkage = g_linkage.copy()

all_clusters = hierarchy.fcluster(g_linkage,g_nclust,criterion='maxclust')

In [ ]:
resting_linkage = g_linkage.copy()

resting_clusters = hierarchy.fcluster(g_linkage,g_nclust,criterion='maxclust')

In [ ]:
passivity_linkage = g_linkage.copy()

passivity_clusters = hierarchy.fcluster(g_linkage,g_nclust,criterion='maxclust')

In [ ]:
long_passivity_linkage = g_linkage.copy()

long_passivity_clusters = hierarchy.fcluster(g_linkage,g_nclust,criterion='maxclust')

In [ ]:
ortho_region_

In [ ]:
np.array(df.region).shape

In [ ]:
regions

In [ ]:
neuron_linkage_clusters

In [ ]:
list(regions)

In [ ]:
region_cmap

In [ ]:
len(neuron_region_cmap)

In [ ]:
region_cmap = sns.color_palette("Paired",12)
# region_map = {r: i for i,r in enumerate(regions)}
neuron_region_cmap = []
for r in df.region_id:
    if r==-1:
        neuron_region_cmap.append((0.5,0.5,0.5))
    else:
        neuron_region_cmap.append(region_cmap[r])

In [ ]:
sns.palplot(sns.color_palette("Paired",12))
plt.xticks(np.arange(12),regions)

In [ ]:
# cm = sb.clustermap(G, row_linkage=g_linkage, row_colors=[g_cmap[x-1] for x in g_clusters],
#         col_linkage=neuron_linkage, col_colors=[neuron_cmap[x-1] for x in neuron_linkage_clusters],
#         method="ward", figsize=(20, 20),cmap="RdBu_r", vmin=-0.1,vmax=0.1)
cm = sb.clustermap(G, row_linkage=g_linkage, row_colors=[[g_cmap[x-1] for x in g_clusters],neuron_region_cmap],
        col_linkage=neuron_linkage, col_colors=[[neuron_cmap[x-1] for x in neuron_linkage_clusters], neuron_region_cmap],
        method="ward", figsize=(20, 20),cmap="RdBu_r", vmin=-0.1,vmax=0.1)

In [ ]:
Z = [0,2,4,6,8,10]
# Z = [2,6,10]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    

In [ ]:
plt.subplots(g_nclust,nZ, figsize=[8*nZ,4*g_nclust])
for c in range(g_nclust):
    clust = c + 1
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(g_clusters==clust) & (df.z==z)].coords
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(g_nclust,nZ,c*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(g_cmap[c][:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Cluster {}, z={}".format(clust,z))

In [ ]:
og2

In [ ]:
og2 = neuron_cmap[2]

In [ ]:
neuron_cmap[2] = (og2[0],og2[1], og2[2]+0.2)

In [ ]:
neuron_cmap[2]
plt.subplots(neuron_nclust,nZ, figsize=[8*nZ,4*neuron_nclust])
for c in range(neuron_nclust):
    clust = c + 1
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(neuron_linkage_clusters==clust) & (df.z==z)].coords
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(neuron_nclust,nZ,c*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(neuron_cmap[c][:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Cluster {}, z={}".format(clust,z))

In [ ]:
u_fish.shape

In [ ]:
plt.plot(u_fish.cpu().numpy())


In [ ]:
plt.figure(figsize=(12,4))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0,0:100,0:100].cpu().detach().numpy(),vmin=0.02,vmax=0.02)
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0,0:100,0:100].cpu().detach().numpy())
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

# plt.subplot(1,3,3)
# plt.imshow(model.B[0:100,0:100].cpu().detach().numpy())
# plt.colorbar(fraction=0.046, pad=0.04)
# plt.title("Bias")
# plt.tick_params(
#     axis='y',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     left=False,      # ticks along the bottom edge are off
#     right=False,         # ticks along the top edge are off
#     labelleft=False) # labels along the bottom edge are off

In [ ]:
plt.figure(figsize=(12*5,4*5))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0].cpu().detach().numpy())
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0].cpu().detach().numpy())
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

# plt.subplot(1,3,3)
# plt.imshow(model.B.cpu().detach().numpy(),vmin=-0.02,vmax=0.02)
# plt.colorbar(fraction=0.046, pad=0.04)
# plt.title("Bias")
# plt.tick_params(
#     axis='y',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     left=False,      # ticks along the bottom edge are off
#     right=False,         # ticks along the top edge are off
#     labelleft=False) # labels along the bottom edge are off

In [ ]:
T.save(model.state_dict(), "granger_raw__TV5000_bz=1_3xl1=1e-0&lr=1e-1_15xl1=5e-2&lr=1e-3.pth")

In [ ]:
def plot_matrix_by_region(W, df, vmin=-0.02, vmax=0.02):
    neuron_map = np.argsort(df.where(df["region_id"]>=0)["region_id"]) # no region is -1
    nHasRegion = np.sum(neuron_map!=-1) + 1 # last -1 index will be removed
    newW = np.zeros(nHasRegion,nHasRegion)
    new_order = neuron_map[np.arange(W.shape[0])]
    new_order = new_order[new_order>=0]
    newW = W[new_order]
    newW = newW[:,new_order]
    fig, ax = plt.subplots(figsize=(20,20))
    im = ax.imshow(newW,vmin=vmin,vmax=vmax)
    fig.colorbar(im)
    return fig

In [ ]:
regions

In [ ]:
df.iloc.head()

In [ ]:
df = df.assign(region_id=pd.Series(np.full(len(df),-1).astype(np.int32)).values)

In [ ]:
for i, region in enumerate(regions):
    idx = np.where(df[region])[0]
    df.loc[idx, "region_id"] = i
df[regions].sum(axis=0)

In [ ]:
plt.imshow()

In [ ]:
plot_matrix_by_region(model.G.data[0].detach().cpu().numpy(),df)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(model.G[0].cpu().detach().numpy(),vmin=-0.02,vmax=0.02)
plt.title("Granger ")
plt.colorbar()

In [ ]:
# Cluster granger by neuron correlation
cm = sb.clustermap(model.G.data[0].detach().cpu().numpy(), row_linkage=neuron_linkage,
                   col_linkage=neuron_linkage, method="ward", figsize=(20, 20),vmin=-0.02,vmax=0.02,)

In [ ]:
# Clustering Granger
G = model.G.data[0].detach().cpu().numpy()
g_linkage = hierarchy.linkage(
    distance.pdist(G), method='ward')

In [ ]:
c, coph_dists = hierarchy.cophenet(g_linkage, distance.pdist(G))
c

In [ ]:
cm = sb.clustermap(G, row_linkage=g_linkage,
                   col_linkage=neuron_linkage, method="ward", figsize=(20, 20),vmin=-0.2,vmax=0.2, cmap="RdBu_r")

In [ ]:
cm = sb.clustermap(G, row_linkage=g_linkage,
                   col_linkage=g_linkage, method="ward", figsize=(20, 20),vmin=-0.2,vmax=0.2)

In [ ]:
g_linkage_clusters = hierarchy.fcluster(g_linkage,5,criterion='maxclust')
# includes all neurons, even those without cluster
cluster_by_g = np.zeros(cnmf.shape[1]+1)
for i,v in enumerate(g_linkage_clusters):
    cluster_by_g[i] = v
cluster_by_g = cluster_by_g[:-1]

In [ ]:
z=9
df, sig = f.get_cnmf_roi_table_and_signals()

In [ ]:
z = 10
#Create a background image by averaging 200 frames and adjusting the gamma.
back_img = np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4)

#Select rois in raphe in this slices, and get their coordinates.
poly_coords = df[(df.z==z)].poly
poly_coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]

#Overlay the ROIs on the background image and display:
img = vizutil.overlay_coords(back_img, poly_coords, [0,0,1], alpha=.5)
plt.figure(figsize=[20,20])
plt.imshow(img,interpolation='nearest')

In [ ]:
g_linkage_clusters = hierarchy.fcluster(g_linkage,4,criterion='maxclust')
# g_linkage_clusters = hierarchy.fcluster(g_linkage, 11, depth=10)
# g_linkage_clusters = hierarchy.fcluster(g_linkage, 5, criterion='distance')
# includes all neurons, even those without cluster
cluster_by_g = np.zeros(cnmf.shape[1]+1)
for i,v in enumerate(g_linkage_clusters):
    cluster_by_g[i] = v
cluster_by_g = cluster_by_g[:-1]
count, hist,_ = plt.hist(cluster_by_g,np.arange(0.5,int(max(cluster_by_g)+1),1))
clusters = np.argwhere(count>100)[:,0]+1
nclust = len(clusters)
print(nclust)

In [ ]:
cluster_by_g

In [ ]:
Z = [0,2,4,6,8,10]
Z = [3,6,9]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    
plt.subplots(nclust,nZ, figsize=[8*nZ,4*nclust])
for c, clust in enumerate(clusters):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        poly_coords = df[(cluster_by_g==clust) & (df.z==z)].poly
        poly_coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nclust,nZ,c*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], poly_coords, [0,0,1], alpha=.5)
        plt.imshow(img,interpolation='nearest')
        plt.title("Cluster {}, z={}".format(clust,z))

# small

In [ ]:

nfeatures = x_fish.shape[1]
nfeatures = 100
model = Granger(nfeatures,1)
X = x_fish[:-1,None,0:nfeatures]
Y = x_fish[1:,0:nfeatures]
data = T.utils.data.TensorDataset(X,Y)
train(model,data,3,l1p=1e-0,lr=1e-1,batch_size=4,verbose=False)
train(model,data,15,l1p=5e-2,lr=1e-3,batch_size=4,verbose=False)

plt.figure(figsize=(12*5,4*5))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0].cpu().detach().numpy())#,vmin=-0.2,vmax=0.2)
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0].cpu().detach().numpy())#,vmin=0.25,vmax=1.5)
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,3)
plt.imshow(model.B.cpu().detach().numpy())#,vmin=-0.1,vmax=0.1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title("Bias")
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off

# archive

In [ ]:

nfeatures = x_fish.shape[1]
nfeatures = 100
model = Granger(nfeatures,1)
X = x_fish[:-1,None,0:nfeatures]
Y = x_fish[1:,0:nfeatures]
data = T.utils.data.TensorDataset(X,Y)
l1 = 1e-6
print("1e-1")
train(model,data,100,l1=l1,batch_size=80,verbose=False)

plt.figure(figsize=(12,4))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0,0:100,0:100].cpu().detach().numpy())
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0,0:100,0:100].cpu().detach().numpy())
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,3)
plt.imshow(model.B[0:100,0:100].cpu().detach().numpy())
plt.colorbar(fraction=0.046, pad=0.04)
plt.title("Bias")
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off

In [ ]:

nfeatures = x_fish.shape[1]
nfeatures = 100
model = Granger(nfeatures,1)
X = x_fish[:-1,None,0:nfeatures]
Y = x_fish[1:,0:nfeatures]
data = T.utils.data.TensorDataset(X,Y)
l1 = 1e-4
print("1e-1")
train(model,data,3,l1=l1,lr=1e-1,batch_size=80,verbose=True)
# print("1e-2")
# train(model,data,10,l1=l1,lr=1e-2,batch_size=80,verbose=True)
print("1e-4")
train(model,data,20,l1=1e-5,lr=1e-3,batch_size=80,verbose=True)

plt.figure(figsize=(12,4))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0,0:100,0:100].cpu().detach().numpy())
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0,0:100,0:100].cpu().detach().numpy())
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,3)
plt.imshow(model.B[0:100,0:100].cpu().detach().numpy())
plt.colorbar(fraction=0.046, pad=0.04)
plt.title("Bias")
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off

In [ ]:
plt.figure(figsize=(12,4))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0,0:100,0:100].cpu().detach().numpy())
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0,0:100,0:100].cpu().detach().numpy())
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,3)
plt.imshow(model.B[0:100,0:100].cpu().detach().numpy())
plt.colorbar(fraction=0.046, pad=0.04)
plt.title("Bias")
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off

In [ ]:

nfeatures = x_fish.shape[1]
nfeatures = 100
model = Granger(nfeatures,1)
X = x_fish[:-1,None,0:nfeatures]
Y = x_fish[1:,0:nfeatures]
data = T.utils.data.TensorDataset(X,Y)
l1 = 1e-3
print("1e-1")
train(model,data,3,l1=l1,lr=1e-1,batch_size=80,verbose=True)
# print("1e-2")
# train(model,data,10,l1=l1,lr=1e-2,batch_size=80,verbose=True)
print("1e-4")
train(model,data,20,l1=1e-6,lr=1e-3,batch_size=80,verbose=True)

plt.figure(figsize=(12,4))
plt.suptitle("G-causality params with L1={}".format(l1))
plt.subplot(1,3,1)
plt.imshow(model.G[0,0:100,0:100].cpu().detach().numpy())
plt.title("Granger ")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,2)
plt.imshow(model.auto[0,0:100,0:100].cpu().detach().numpy())
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title("Autocorrelation")
plt.colorbar(fraction=0.046, pad=0.04)

plt.subplot(1,3,3)
plt.imshow(model.B[0:100,0:100].cpu().detach().numpy())
plt.colorbar(fraction=0.046, pad=0.04)
plt.title("Bias")
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off

In [ ]:
cm.data = np.zeros([5000,5000])

In [ ]:
cm.fig.set_canvas

In [ ]:
cm.fig

In [ ]:
import skvideo.io
import gc

In [ ]:
gc.collect()

In [ ]:
2

In [ ]:
winsize = 250
overlap = 0.9
start = 0
end = neurons.shape[0]-winsize
print(end/(winsize*(1-overlap))*4/25)

In [ ]:
neuron_corr_colors = [neuron_cmap[x-1] for x in neuron_linkage_clusters]

In [ ]:
def fig_to_array(fig):
    "Convert matplotlib fig into a numpy array (rgb image)"
    fig.canvas.draw()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return data

In [ ]:
winsize = 250
overlap = 0.9
start = 0
end = neurons.shape[0]-winsize
neuron_corr_colors = [neuron_cmap[x-1] for x in neuron_linkage_clusters]
# use single hierarchy
writer = skvideo.io.FFmpegWriter("long_color_corr_over_time.mp4", outputdict={'-b': '30000000'})

pbar = tqdm(total=end/(winsize*(1-overlap)))
while start < end:
    corr = corrcoef(neurons[start:start+winsize].t()).cpu().numpy()
    cm = sb.clustermap(corr, row_linkage=neuron_linkage, col_linkage=neuron_linkage, method="ward",
            col_colors=neuron_corr_colors, row_colors=neuron_corr_colors,
            figsize=(15, 15))
    f = fig_to_array(cm.fig)
    for i in range(4):
        writer.writeFrame(f)
    start += int((winsize*(1-overlap)))
    pbar.update(1)
    plt.close('all')
    gc.collect()
pbar.close()
writer.close()

In [ ]:
corr = corrcoef(neurons.t()).cpu().numpy()
cm = sb.clustermap(corr, row_linkage=neuron_linkage, col_linkage=neuron_linkage, method="ward",
            col_colors=neuron_corr_colors, row_colors=neuron_corr_colors,
            figsize=(15, 15))